### 数据(鸢尾数据的类别预测)
    特征:
        sepal length（花萼长度）
        sepal width（花萼宽度）
        petal length（花瓣长度）
        petal width（花瓣宽度）
    结果:
        Iris-setosa,Iris-versicolor,Iris-virginica

    #数据表创建
        drop table if exists Iris_train;
        drop table if exists Iris_eval;
        create table Iris_train(
            id bigserial,
            sepalH float8,
            sepalW float8,
            petalH float8,
            petalW float8,
            label integer
        );
        create table Iris_eval(
            id bigserial,
            sepalH float8,
            sepalW float8,
            petalH float8,
            petalW float8,
            label integer
        );

In [13]:
##数据录入
import pandas
import numpy as np

df=pandas.read_csv('data/iris.data',names=['sepalH','sepalW','petalH','petalW','label'])
df.head()

train_sqls,eval_sqls='',''
trainindex=np.random.choice(len(df),size=180)

labelfunc={'Iris-setosa':0,'Iris-versicolor':1,'Iris-virginica':2}
for i in range(len(df)):
    d=df.iloc[i]
    if i in trainindex:
        sql="insert into Iris_train(sepalH,sepalW,petalH,petalW,label) values('%.2f','%.2f','%.2f','%.2f','%s');"%(d['sepalH'],d['sepalW'],d['petalH'],d['petalW'],labelfunc[d['label']])
        train_sqls=train_sqls+sql+'\n'
    else:
        sql="insert into Iris_eval(sepalH,sepalW,petalH,petalW,label) values('%.2f','%.2f','%.2f','%.2f','%s');"%(d['sepalH'],d['sepalW'],d['petalH'],d['petalW'],labelfunc[d['label']])
        
        eval_sqls=eval_sqls+sql+'\n'
with open('data/iris_train.sql','w') as fs:
    fs.write(train_sqls)
with open('data/iris_eval.sql','w') as fs:
    fs.write(eval_sqls)
    

### 使用multiNormial 在 训练数据集上面 "训练"
http://madlib.apache.org/docs/latest/group__grp__multinom.html

    select madlib.multinom(
             'Iris_train',  --源表
             'Iris_train_param', --生成参数表
             'label', --label,因变量
             'array[sepalH,sepalW]',--自变量,只选择前2个变量
             '0',
             'logit'
            );
    select * from Iris_train_param;
<img src='img/cl1.png' />
def train(param:Map):
    param:
        src:String
        label:String
        features:String[]
        ref_category:String
    return 
        

In [ ]:
class MultiClassifier():
    def __init__(taskId):
        self.taskId
    def train(param:dict):
        '''
            param包含如下字段,分别对于sql函数的参数
                    src:String
                    label:String
                    features:String[]
                    ref_category:String
                    task:String 任务表名称 
        1.根据taskId,src源表名,本分类器名,生成tgt表名
        2.根据参数param,调用相应的方法multino
        3.方法执行完成后,要把"任务记录表"中taskId=self.taskId的state置为finish
        4.程序报错要捕捉,并把捕捉信息记录到 "任务记录表" taskId=self.taskId的error info字段
        
        返回:dict
            传入的param里面添加2个字段
                state:String success,fail
                error:String,有错显示捕捉的错误
                model:生成的tgt模型表
        '''
    def predict(param:dict):
        '''
            param:
                src:String 输入源表名
                model:String,模型参数名
                response:
            
            根据src,model,taskId生成输出表名,执行multinom_predict方法.
            
            返回:dict
                state:
                error:
                out:String 输出的表
        '''

$\beta1=[6.45,-11.47]$,$\beta2=[6.94,-12.43]$

$score1=\beta1^T *x$ 

$score2=\beta2^T *x$

$P(x==1)=\frac{exp(score1)}{exp(score1)+exp(score2)+1}$

$P(x==2)=\frac{exp(score2)}{exp(score1)+exp(score2)+1}$

$P(x==3)=\frac{1}{exp(score1)+exp(score2)+1}$

### 在 "测试数集" 预测

    select madlib.multinom_predict(
                     'Iris_train_param',  --模型参数表
                     'iris_eval',     --输入要预测数据
                     'iris_eval_result',  --输出结果
                     'response' --输出类型,response表示只输出结果,probability输出所有的概率
                    );

    select * from iris_eval_result --查看结果
<img src='img/cl2.png'>

    select avg(cast(r as int)) from (select (a.label=cast(b.category as int)) as r  from iris_eval a,iris_eval_result b where a.id=b.id) t; --查看准且率
<img src='img/cl3.png'>